In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import argparse
import os, shutil
from sklearn.model_selection import train_test_split

In [2]:
img_dir = '/data/handhygiene/jpg/'
excel_path = '/data/handhygiene/hh_label.xlsx'
csv_path = '/data/handhygiene/hh_label.csv'
test_size= 0.1
val_size = 0.1

df = pd.read_csv(csv_path)

In [3]:
sub_dirs = [x[0] for x in os.walk(img_dir)][1:]

In [4]:
def split_train_val_test(img_dir, vids):
    x_train, x_test = train_test_split(vids, test_size=test_size)
    x_train, x_val = train_test_split(x_train, test_size=val_size)
     
    print(x_train.values, x_val.values, x_test.values)
    for subdir in sub_dirs:
        for phase in ["train", "val", "test"]:
            target_dir = os.path.join(img_dir, phase)
            if not os.path.exists(target_dir):
                os.mkdir(target_dir)

            if phase == "train":
                target = x_train
            elif phase == "val":
                target = x_val
            elif phase == "test":
                target = x_test

            for video_id in tqdm(target):
                video_id=video_id
                origin_dir = os.path.join(subdir, str(video_id)+'_*')
                dst_dir = os.path.join(target_dir, os.path.basename(subdir))
                if not os.path.exists(dst_dir):
                    os.mkdir(dst_dir)
                os.system("mv %s %s" % (origin_dir, dst_dir))
                print(origin_dir, dst_dir)

            # create empty dataframe
            df_new = df[df['video_id'] == 0]
            for i in target:
                add= df[df['video_id'] == i]
                df_new = pd.concat([df_new, pd.DataFrame(add)])

            target_path = csv_path.split('.')[-2]
            dest = "%s_%s.csv" % (target_path, phase)
            df_new.to_csv(dest, index=False)
            
        os.rmdir(subdir)
    return

In [5]:
split_by = df['video_id'].drop_duplicates()
split_train_val_test(img_dir, split_by)

  0%|          | 0/5 [00:00<?, ?it/s]

(array([13, 11,  5,  8, 18]), array([17]), array([2]))
('/data/handhygiene/jpg/notclean/13_*', '/data/handhygiene/jpg/train/notclean')
('/data/handhygiene/jpg/notclean/11_*', '/data/handhygiene/jpg/train/notclean')
('/data/handhygiene/jpg/notclean/5_*', '/data/handhygiene/jpg/train/notclean')
('/data/handhygiene/jpg/notclean/8_*', '/data/handhygiene/jpg/train/notclean')
('/data/handhygiene/jpg/notclean/18_*', '/data/handhygiene/jpg/train/notclean')
('/data/handhygiene/jpg/notclean/17_*', '/data/handhygiene/jpg/val/notclean')
('/data/handhygiene/jpg/notclean/2_*', '/data/handhygiene/jpg/test/notclean')
('/data/handhygiene/jpg/clean/13_*', '/data/handhygiene/jpg/train/clean')
('/data/handhygiene/jpg/clean/11_*', '/data/handhygiene/jpg/train/clean')
('/data/handhygiene/jpg/clean/5_*', '/data/handhygiene/jpg/train/clean')
('/data/handhygiene/jpg/clean/8_*', '/data/handhygiene/jpg/train/clean')
('/data/handhygiene/jpg/clean/18_*', '/data/handhygiene/jpg/train/clean')


100%|██████████| 1/1 [00:00<00:00, 62.26it/s]

('/data/handhygiene/jpg/clean/17_*', '/data/handhygiene/jpg/val/clean')
('/data/handhygiene/jpg/clean/2_*', '/data/handhygiene/jpg/test/clean')
